## This compares to DoubleBracketIteration whenever possible

Note that we are not using the reduced group commutator so just 4 steps of the iteration using circuit representations uses quite a lot of time (several seconds)

In [1]:
import inspect
import sys
sys.path.append("../../tests")
from test_models_dbi import *
def print_function_source_code( func ):
    out = inspect.getsourcelines(func)  
    from functools import reduce
    print(reduce(str.__add__, out[0]))
import qibo
backend = qibo.backends.construct_backend("numpy")
qibo.set_backend("numpy")
nqubits = 3

[Qibo 0.2.7|INFO|2024-05-03 09:28:36]: Using numpy backend on /CPU:0


In [105]:
r=0.01
mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
h_x = SymbolicHamiltonian(
        symbols.X(0)
    + symbols.Z(0) * symbols.X(1)
    + symbols.Y(2)
    + symbols.Y(1) * symbols.Y(2),
nqubits=3,
)
d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits=3)
h_input = h_x + d_0

dbi = DoubleBracketIteration(deepcopy(h_input.dense))

evolution_oracle = EvolutionOracle(h_input, "ZX", mode_evolution_oracle)   
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))
gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_reduced
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
   mode_evolution_oracle)
evolution_oracle.eps_trottersuzuki = 1e-1
from numpy.linalg import norm

for _ in range(4):
    dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.group_commutator )
    gci(r, diagonal_association= evolution_oracle_diagonal_target )

k_r = dbi.h.matrix
j_r = gci.h.matrix




[Qibo 0.2.7|WARNING|2024-05-03 09:52:46]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-03 09:52:46]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [106]:
u = gci.iterated_hamiltonian_evolution_oracle.get_composed_circuit()

In [107]:
q = u.queue

In [108]:
for g in q:
    print(g.name)

Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary
Unitary


In [109]:
it = 0
for g in u.queue:
    if g.matrix().shape[0] == 4:
        it = it+1

In [110]:
it

1932

In [111]:
h0 = h_input.dense.matrix
h_end = u.unitary().conj().T @ h0 @ u.unitary()

In [112]:
norm(h_end - dbi.h.matrix)

0.0011754492342204688

In [ ]:
gci.input_hamiltonian_evolution_oracle.eps_trottersuzuki

In [ ]:
norm(h_end - dbi.h.matrix)/gci.input_hamiltonian_evolution_oracle.eps_trottersuzuki

We conclude that this is fine. Feel free to change to the numerical mode where the error will start piling up merely from the floating point operations and will build up from the double digit precision.

In [ ]:
c= qibo.Circuit(nqubits=4)
c.add(gates.CNOT(1,3))
c.add(gates.Z(2))

In [ ]:
g = c.queue[0]

In [27]:
g.target_qubits

(3,)

In [29]:
g

In [30]:
for g in c.queue:
    if isinstance(g,qibo.gates.gates.CNOT):
        TNS.apply_CNOT(g.target_qubits,g.control_qubits)

NameError: name 'TNS' is not defined